In [21]:
# ZIA GitHub Enrichment (NO API Required): this scrip parses GitHub repo URLs from ../data/lab_github.csv, groups by owner/repo,
# then scrape each repo homepage to pull stars, watching, forks, last commit date, and README text.
#
# Works with ZIA URLs that often lack a scheme (e.g., "github.com/org/repo").
# Avoids false positives from URLs where "github.com/..." appears inside another site's query string.
#
# Outputs:
#   - repo_counts: distinct repos + proxy_hits
#   - repo_enrichment: repo_counts + scraped metadata
#   - repos_enriched: row-level subset of df that includes repo URLs + metadata

import re
import time
import pandas as pd
import requests
from urllib.parse import urlparse, unquote

# ----------------------------
# Config
# ----------------------------
CSV_PATH = "../data/lab_github.csv"
SLEEP_SECONDS = 0.6          # throttling to GitHub; reduce if tiny list, increase if big list
MAX_README_CHARS = 8000

ALLOWED_HOSTS = {
    "github.com", "www.github.com",
    "raw.githubusercontent.com",
    "codeload.github.com",
    "objects.githubusercontent.com",
}

# Show full text in DataFrame cells (can get huge)
pd.set_option("display.max_colwidth", None)

# Optional: show more rows/columns too
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

# ----------------------------
# Helpers
# ----------------------------
def normalize_url(u: str) -> str:
    u = (u or "").strip()
    if not u:
        return ""
    if "://" in u:
        return u
    return "https://" + u

def to_int_compact(s: str):
    """Convert '243k'/'6.7k'/'50.6k'/'1,234' => int."""
    if not s:
        return None
    s = s.strip().lower().replace(",", "")
    m = re.match(r"^(\d+(?:\.\d+)?)([km])?$", s)
    if not m:
        return None
    num = float(m.group(1))
    suf = m.group(2)
    if suf == "k":
        num *= 1_000
    elif suf == "m":
        num *= 1_000_000
    return int(num)

def clean_ws(text: str):
    return re.sub(r"\s+", " ", text or "").strip()

def extract_owner_repo_strict(raw_url: str):
    """
    Extract owner/repo only if the URL's actual host is GitHub-related.
    Prevents false positives from google.com URLs that contain "github.com/..." in query parameters.
    Returns: (owner, repo, owner_repo, host) or (None,...)
    """
    u = normalize_url(raw_url)
    if not u:
        return (None, None, None, None)

    try:
        p = urlparse(u)
    except Exception:
        return (None, None, None, None)

    host = (p.netloc or "").lower()
    if host not in ALLOWED_HOSTS:
        return (None, None, None, None)

    path = unquote(p.path or "").strip("/")
    parts = [x for x in path.split("/") if x]
    if len(parts) < 2:
        return (None, None, None, host)

    owner, repo = parts[0].strip(), parts[1].strip()

    # Strip .git
    if repo.lower().endswith(".git"):
        repo = repo[:-4]

    # Trim punctuation
    repo = repo.rstrip(").,;")

    owner = owner.lower()
    repo = repo.lower()

    # Filter obvious non-repo routes for github.com
    non_repo = {"settings","pricing","topics","marketplace","collections","features","about","site","login","join","organizations","search","apps"}
    if host in {"github.com","www.github.com"} and owner in non_repo:
        return (None, None, None, host)

    # Basic repo/owner sanity: typical GitHub charset
    if not re.match(r"^[a-z0-9_.-]+$", owner) or not re.match(r"^[a-z0-9_.-]+$", repo):
        return (None, None, None, host)

    return (owner, repo, f"{owner}/{repo}", host)

def find_anchor_block(html: str, owner_repo: str, endpoint: str):
    """
    Return the full <a ...>...</a> block where href="/{owner_repo}/{endpoint}" (optionally with query params).
    """
    pattern = (
        rf'(<a\b[^>]*href="/{re.escape(owner_repo)}/{endpoint}(?:\?[^"]*)?"[^>]*>'
        rf'[\s\S]*?</a>)'
    )
    m = re.search(pattern, html, re.IGNORECASE)
    return m.group(1) if m else None

def extract_count_from_anchor(anchor_html: str):
    """
    Extract the metric count from inside an <a>...</a> block on GitHub.
    Supports layouts like:
      - <span class="Counter">243k</span>
      - <strong>6.7k</strong>
    Avoids pulling numbers from SVG icons and path data.
    """
    if not anchor_html:
        return None

    # Remove svg blocks to prevent matching numbers from svg path data
    cleaned = re.sub(r"<svg[\s\S]*?</svg>", " ", anchor_html, flags=re.IGNORECASE)

    # 1) Counter span
    m = re.search(r'class="[^"]*\bCounter\b[^"]*"[^>]*>\s*([^<]+?)\s*<', cleaned, re.IGNORECASE)
    if m:
        val = to_int_compact(m.group(1))
        if val is not None:
            return val

    # 2) <strong>243k</strong> / <strong>6.7k</strong>
    m = re.search(r"<strong[^>]*>\s*([^<\s]+)\s*</strong>", cleaned, re.IGNORECASE)
    if m:
        val = to_int_compact(m.group(1))
        if val is not None:
            return val

    # 3) aria-label fallback if present
    m = re.search(r'aria-label="([^"]+)"', cleaned, re.IGNORECASE)
    if m:
        mnum = re.search(r'(\d+(?:\.\d+)?[kKmM]?)', m.group(1))
        if mnum:
            val = to_int_compact(mnum.group(1))
            if val is not None:
                return val

    # 4) Last resort: number in visible text only (after stripping tags)
    # (still anchored to this <a> block, not the whole page)
    text_only = re.sub(r"<[^>]+>", " ", cleaned)
    text_only = clean_ws(text_only)
    m = re.search(r"\b(\d+(?:\.\d+)?[kKmM]?)\b", text_only)
    if m:
        val = to_int_compact(m.group(1))
        if val is not None:
            return val

    return None

def scrape_repo(owner_repo: str, session: requests.Session, sleep_s: float = SLEEP_SECONDS):
    """
    Scrape https://github.com/{owner_repo} for:
      - stars, watching, forks (from link anchors)
      - last_commit_date (relative-time datetime)
      - readme_text (markdown-body article)
    """
    url = f"https://github.com/{owner_repo}"
    out = {
        "owner_repo": owner_repo,
        "repo_url": url,
        "scrape_ok": False,
        "status_code": None,
        "stars": None,
        "watching": None,
        "forks": None,
        "last_commit_date": None,
        "readme_text": None,
    }

    try:
        r = session.get(url, timeout=30, allow_redirects=True)
        out["status_code"] = r.status_code
        html = r.text or ""
    except Exception:
        return out

    time.sleep(sleep_s)

    blocked_signals = ["Sign in to GitHub", "Verify", "rate limit", "Something went wrong"]
    if r.status_code != 200 or any(sig.lower() in html.lower() for sig in blocked_signals):
        return out

    out["scrape_ok"] = True

    # Counts: parse within the correct anchor only
    star_a  = find_anchor_block(html, owner_repo, "stargazers")
    watch_a = find_anchor_block(html, owner_repo, "watchers")
    fork_a  = find_anchor_block(html, owner_repo, "forks")

    out["stars"] = extract_count_from_anchor(star_a)
    out["watching"] = extract_count_from_anchor(watch_a)
    out["forks"] = extract_count_from_anchor(fork_a)

    # Commit date (best effort)
    m = re.search(r'Latest commit[\s\S]{0,9000}?<relative-time[^>]+datetime="([^"]+)"', html, re.IGNORECASE)
    if m:
        out["last_commit_date"] = m.group(1)
    else:
        m = re.search(r'<relative-time[^>]+datetime="([^"]+)"', html, re.IGNORECASE)
        if m:
            out["last_commit_date"] = m.group(1)

    # README text (best effort)
    readme_block = None
    m = re.search(
        r'id="readme"[\s\S]{0,25000}?<article[^>]*class="[^"]*markdown-body[^"]*"[^>]*>([\s\S]*?)</article>',
        html, re.IGNORECASE
    )
    if m:
        readme_block = m.group(1)
    else:
        m = re.search(
            r'<article[^>]*class="[^"]*markdown-body[^"]*"[^>]*>([\s\S]*?)</article>',
            html, re.IGNORECASE
        )
        if m:
            readme_block = m.group(1)

    if readme_block:
        readme_block = re.sub(r"<(script|style)[^>]*>[\s\S]*?</\1>", " ", readme_block, flags=re.IGNORECASE)
        text = re.sub(r"<[^>]+>", " ", readme_block)
        text = (text.replace("&amp;", "&")
                    .replace("&lt;", "<")
                    .replace("&gt;", ">")
                    .replace("&quot;", '"')
                    .replace("&#39;", "'"))
        text = clean_ws(text)
        out["readme_text"] = text[:MAX_README_CHARS] if text else None

    return out

# ----------------------------
# Main
# ----------------------------
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip().lower() for c in df.columns]
df["url"] = df["url"].astype(str).fillna("").str.strip()

df[["gh_owner","gh_repo","owner_repo","gh_host"]] = df["url"].apply(lambda x: pd.Series(extract_owner_repo_strict(x)))

repos_df = df.dropna(subset=["owner_repo"]).copy()

print("Total rows:", len(df))
print("Rows with extracted owner/repo:", len(repos_df))
print("Distinct repos:", repos_df["owner_repo"].nunique())

repo_counts = (
    repos_df.groupby("owner_repo", as_index=False)
            .size()
            .rename(columns={"size":"proxy_hits"})
            .sort_values("proxy_hits", ascending=False)
)
display(repo_counts.head(30))

# Session headers
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
})

# Quick sanity test on facebook/react
test = scrape_repo("facebook/react", session=session, sleep_s=0.0)
print("Test scrape facebook/react:", test)

# Enrich all repos
distinct_repos = repo_counts["owner_repo"].astype(str).tolist()

enriched_rows = []
for idx, owner_repo in enumerate(distinct_repos, start=1):
    enriched_rows.append(scrape_repo(owner_repo, session=session, sleep_s=SLEEP_SECONDS))
    if idx % 25 == 0:
        print(f"Enriched {idx}/{len(distinct_repos)} repos...")

enriched = pd.DataFrame(enriched_rows)
repo_enrichment = repo_counts.merge(enriched, on="owner_repo", how="left")

display(repo_enrichment.sort_values(["proxy_hits","stars"], ascending=[False, False]).head(50))

# Optional: join back to row-level repo URL events
repos_enriched = repos_df.merge(repo_enrichment, on="owner_repo", how="left")
display(repos_enriched[["url","gh_host","owner_repo","proxy_hits","stars","watching","forks","last_commit_date"]].head(25))


Total rows: 8012
Rows with extracted owner/repo: 743
Distinct repos: 8


,owner_repo,proxy_hits
0,anthropics/skills,204
1,datatalksclub/data-engineering-zoomcamp,183
2,facebook/react,153
3,googlecloudplatform/terraformer,137
7,tensorflow/tensorflow,53
6,public-apis/public-apis,10
4,peass-ng/peass-ng,2
5,publi-apis/public-apis,1


Test scrape facebook/react: {'owner_repo': 'facebook/react', 'repo_url': 'https://github.com/facebook/react', 'scrape_ok': True, 'status_code': 200, 'stars': 243000, 'watching': 6700, 'forks': 50600, 'last_commit_date': '2026-01-26T18:29:43Z', 'readme_text': 'React · React is a JavaScript library for building user interfaces. Declarative: React makes it painless to create interactive UIs. Design simple views for each state in your application, and React will efficiently update and render just the right components when your data changes. Declarative views make your code more predictable, simpler to understand, and easier to debug. Component-Based: Build encapsulated components that manage their own state, then compose them to make complex UIs. Since component logic is written in JavaScript instead of templates, you can easily pass rich data through your app and keep the state out of the DOM. Learn Once, Write Anywhere: We don\'t make assumptions about the rest of your technology stack, 

owner_repo  proxy_hits  \
0                        anthropics/skills         204   
1  datatalksclub/data-engineering-zoomcamp         183   
2                           facebook/react         153   
3          googlecloudplatform/terraformer         137   
4                    tensorflow/tensorflow          53   
5                  public-apis/public-apis          10   
6                        peass-ng/peass-ng           2   
7                   publi-apis/public-apis           1   

                                                     repo_url  scrape_ok  \
0                        https://github.com/anthropics/skills       True   
1  https://github.com/datatalksclub/data-engineering-zoomcamp       True   
2                           https://github.com/facebook/react       True   
3          https://github.com/googlecloudplatform/terraformer       True   
4                    https://github.com/tensorflow/tensorflow       True   
5                  https://github.com/public-apis/public-apis      False   
6                        https://github.com/peass-ng/peass-ng       True   
7                   https://github.com/publi-apis/public-apis      False   

   status_code     stars  watching    forks      last_commit_date  \
0          200   71200.0     530.0   7300.0                  None   
1          200   38500.0     564.0   7700.0                  None   
2          200  243000.0    6700.0  50600.0  2026-01-26T18:29:43Z   
3          200   14500.0     158.0   1800.0  2025-03-26T23:37:41Z   
4          200  194000.0    7400.0  75200.0  2025-08-13T17:45:14Z   
5          200       NaN       NaN      NaN                  None   
6          200   19300.0     235.0   3300.0  2026-02-12T19:35:05Z   
7          404       NaN       NaN      NaN                  None   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

,url,gh_host,owner_repo,proxy_hits,stars,watching,forks,last_commit_date
0,github.com/anthropics/skills,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
1,github.com/anthropics/skills/hovercards/citation/sidebar_partial?tree_name=main,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
2,github.com/anthropics/skills/used_by_list,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
3,github.com/anthropics/skills/refs?type=branch,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
4,github.com/anthropics/skills/tree-commit-info/main,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
5,github.com/anthropics/skills/branch-and-tag-count,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
6,github.com/anthropics/skills/latest-commit/main,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
7,github.com/anthropics/skills/archive/refs/heads/main.zip,github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
8,codeload.github.com/anthropics/skills/zip/refs/heads/main,codeload.github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
9,codeload.github.com/anthropics/skills/zip/refs/heads/main,codeload.github.com,anthropics/skills,204,71200.0,530.0,7300.0,None
